In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler


import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
from mlflow.server import get_app_client
from mlflow.server.auth.client import AuthServiceClient

# from bulkboto3 import BulkBoto3
from dotenv import load_dotenv
load_dotenv()

import yaml
import boto3
import pickle
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

os.environ['AWS_ACCESS_KEY_ID'] = "admin" # логин от s3
os.environ['AWS_SECRET_ACCESS_KEY'] = "23wesdxc" # пароль от s3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.229.8.165:8000" # адрес s3
os.environ['MLFLOW_TRACKING_USERNAME'] = "admin" # логин MLFlow
os.environ['MLFLOW_TRACKING_PASSWORD'] = "23wesdxc" # пароль MLFlow 

# Подключение к MLFlow
mlflow.set_tracking_uri(uri="http://10.229.8.165:8080")

# # Подключение к s3
# session = boto3.session.Session()
# s3 = session.client(
#     service_name='s3',
#     endpoint_url=os.environ['MLFLOW_S3_ENDPOINT_URL']
# )

config = yaml.safe_load(open("./config.yaml"))

ModuleNotFoundError: No module named 'mlflow'

# s3

In [ ]:
session = boto3.session.Session()
s3 = session.client(service_name="s3", endpoint_url="http://10.229.8.165:8000")

In [ ]:
# Список бакетов
s3.list_buckets()["Buckets"]

[{'Name': 'data-storage',
  'CreationDate': datetime.datetime(2025, 1, 27, 3, 6, 50, 543000, tzinfo=tzutc())},
 {'Name': 'mlflow-bucket',
  'CreationDate': datetime.datetime(2025, 1, 27, 3, 5, 35, 589000, tzinfo=tzutc())}]

In [ ]:
# Показать файлы в бакете
bucket_name = "data-storage"

for key in s3.list_objects(Bucket=bucket_name)["Contents"]:
    print(key["Key"])

sales-prdict/files/md5/08/627c764eca5865daa5d571bfc0212b
sales-prdict/files/md5/08/890787676cacd7220c50d22cc767e8
sales-prdict/files/md5/09/abaa3dd779abd5e8b1a0b7789b23dc
sales-prdict/files/md5/15/5f8139ddf794a44a219c08962373e5
sales-prdict/files/md5/1b/500a059053e6987609632281ce82bf
sales-prdict/files/md5/5c/93426469e642198413a5ab942b002b
sales-prdict/files/md5/63/15056c6be10d94783424d1bbc4531e
sales-prdict/files/md5/64/bb69506a28500156edfa2aecb1ee55
sales-prdict/files/md5/68/39d616157acd1543b79317ca71a8ca
sales-prdict/files/md5/6a/18dc43b61949df98458b5c8e174a1c
sales-prdict/files/md5/6d/94ec66d00a28ea7043ea46e78fd341
sales-prdict/files/md5/8e/be1d40d330553942e6f50a2f051c9c
sales-prdict/files/md5/91/14abb25908017d6b5774fd6af9eb66
sales-prdict/files/md5/a1/1b8ba67641b03d0c2525e813e99498
sales-prdict/files/md5/a4/b1183bb077d328857ec2b03d89c4f5
sales-prdict/files/md5/af/8e7eb07ae3b935bb324b29845b8b9f
sales-prdict/files/md5/b1/417254ba7eba061083e7005f9d12db.dir
sales-prdict/files/md5/b6/4

## Администрирование MLFlow

In [ ]:
from mlflow.server.auth.client import AuthServiceClient

os.environ['MLFLOW_TRACKING_USERNAME'] = "admin" # логин MLFlow
os.environ['MLFLOW_TRACKING_PASSWORD'] = "imk14f53" # пароль MLFlow
client = AuthServiceClient(os.getenv("MLFLOW_TRACKING_URI"))

In [ ]:
user = client.get_user("admin")
print(f"user_id: {user.id}")
print(f"username: {user.username}")
print(f"password_hash: {user.password_hash}")
print(f"is_admin: {user.is_admin}")

MlflowException: API request to endpoint /api/2.0/mlflow/users/get failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'

## Администрирование MLFlow

In [3]:
client = AuthServiceClient(os.getenv("MLFLOW_TRACKING_URI"))

In [11]:
client.create_experiment_permission(
    experiment_id="1",
    username="qwer",
    permission="EDIT"
)

In [ ]:
# Создание нового пользователя
client.create_user("newuser", "newpassword")

# Удаление пользователя (возможно только после удаления всех доступов)
client.delete_user("newuser")

# Обновление пароля
client.update_user_password("admin", "testpass")

# Создать привелегии
client.create_experiment_permission(
    experiment_id="1",
    username="newuser",
    permission="EDIT" # READ / MANAGE https://mlflow.org/docs/latest/auth/index.html#how-it-works
)

# Обновить привелегии
client.update_experiment_permission(
    experiment_id="1",
    username="newuser",
    permission="MANAGE"
)

# Отозвать привелегии
client.delete_experiment_permission(
    experiment_id="1",
    username="newuser"
)

# Админские права
client.update_user_admin(
    username="newuser",
    is_admin=True
)

In [5]:
user = client.get_user("newuser")
print(f"user_id: {user.id}")
print(f"username: {user.username}")
print(f"password_hash: {user.password_hash}")
print(f"is_admin: {user.is_admin}")

user_id: 4
username: newuser
password_hash: REDACTED
is_admin: False


In [6]:
ep = client.get_experiment_permission("1", "newuser")
print(f"experiment_id: {ep.experiment_id}")
print(f"user_id: {ep.user_id}")
print(f"permission: {ep.permission}")

RestException: RESOURCE_DOES_NOT_EXIST: Experiment permission with experiment_id=1 and username=newuser not found